In [1]:
import os
import re
from scipy.io import loadmat, savemat
import numpy as np
from pprint import pp
import shutil
import tempfile
import uuid
from pathlib import Path
import csv
import datetime

In [2]:
# GLOBAL CONSTANTS - DO NOT CHANGE THIS SECTION -

LASTCHANGE_STRING = "2024-08-20_mir_flip_zaxes"

# Matrix tranform to flip x,y axes and negate z axis
# this is multiplied by each 3d point and by each rotation matrix
transform = np.array([[0, 1, 0], [ 1, 0, 0] , [ 0, 0, -1]])


In [3]:
def save_backup_samedir(src_filepath):
    """For each file, create a copy of it with a .old postfix in the same directory"""
    assert Path(src_filepath).exists(), "src_filepath must exist"
    src_filepath_str = str(Path(src_filepath).resolve())

    candidate_path = src_filepath
    
    # try 1000 times to make a unique filename then throw an error if it failed
    tries = 0
    while True:
        candidate_path = Path(f"{candidate_path}.old")
        if candidate_path.exists():
            tries += 1
        else:
            break

        if tries > 1000: 
            raise Exception("Unable to generate unique file name. Tried 1000 times already.")

    dest_filepath = candidate_path
    
    shutil.copy2(src_filepath, dest_filepath)
    print(f"BACKUP src={src_filepath} dest={dest_filepath}")


def update_hires_file(filepath, saveto=None):    
    print(f"update_hires_file: {filepath}")    
    
    mat1 = loadmat(filepath)
    
    if 'lastchange' in mat1 and mat1['lastchange'] == LASTCHANGE_STRING:
        raise Exception("SKIPPING BECAUSE UPDATE WAS ALREADY APPLIED")

    save_backup_samedir(filepath)

    
    mat1['lastchange'] = LASTCHANGE_STRING
    mat1['r'] =transform @ mat1['r']

    if saveto is not None:
        print(f"Saving update mat file as: {saveto}")
        savemat(saveto, mat1)
    else:
        print("Not saving file because saveto not specified")


def update_label3d_working_file(filepath, saveto=None):
    """Fix label3d save files (these can be loaded in label3d). This is NOT 
    the label3d exported files used to train dannce (those are *dannce.mat)"""
    
    mat3 = loadmat(filepath)
    
    if 'lastchange' in mat3 and mat3['lastchange'] == LASTCHANGE_STRING:
        raise Exception("SKIPPING BECAUSE UPDATE WAS ALREADY APPLIED")


    save_backup_samedir(filepath)

    
    n_cams = mat3['camParams'].shape[0]
    n_samples = mat3['data_3D'].shape[0]
    n_joints =  mat3['data_3D'].shape[1]//3
    
    
    # update 3d points (labeld 3d-space)
    # format is [x y z x2 y2 z2 ... xN yN zN] for each row
    # mat3d['data_3D'].shape = (75,69)
    dat = mat3['data_3D'] # old shape: (75, 69)
    dat = dat.reshape(n_samples, n_joints, 3) # new shape: (75, 23, 3)
    print("fixing all 3d points for #samples={n_samples}")
    for joint_idx in range(n_joints):
        print(f"updating on joint_idx={joint_idx}")
        joint_dat = dat[:,joint_idx,:] #shape: (75,3)
        joint_dat = (transform @ joint_dat.T).T
        dat[:,joint_idx,:] = joint_dat
    mat3['data_3D'] = dat.reshape(n_samples, 3*n_joints)
    
    # update camera parameters (rotation mtx) for each camera
    # mat3['camParams'].shape = (6,1)
    for cam_idx in range(n_cams):
        print(f"updating rot mtx for cam_idx={cam_idx}")
        
        this_rot_mtx = mat3['camParams'][cam_idx,0]['r'][0,0]
        this_rot_mtx = transform @ this_rot_mtx
        mat3['camParams'][cam_idx,0]['r'][0,0] = this_rot_mtx
    
    mat3['lastchange'] = LASTCHANGE_STRING

    if saveto is not None:
        print(f"Saving update mat file as: {saveto}")
        savemat(saveto, mat3)
    else:
        print("Not saving file because saveto not specified")


def update_calib_for_dannce(filepath,saveto=None):
    print(f"update_dannce_mat_file: {filepath}")

    mat4 = loadmat(filepath)
    
    if 'lastchange' in mat4 and mat4['lastchange'] == LASTCHANGE_STRING:
        raise Exception("SKIPPING BECAUSE UPDATE WAS ALREADY APPLIED")

    save_backup_samedir(filepath)
    
    mat4['lastchange'] = LASTCHANGE_STRING

    n_cams = mat4['camnames'].shape[0]
    
    # n_cams = mat4['labelData'].shape[0]
    # n_samples = mat4['labelData'][0,0][0,0]['data_3d'].shape[0]
    # n_joints =  mat4['labelData'][0,0][0,0]['data_3d'].shape[1]//3
    
    # print(f"n_cams={n_cams}, n_samples={n_samples}, n_joints={n_joints}")
    
    # update camera parameters (rotation mtx) for each camera
    # mat4['camParams'].shape = (6,1)
    for cam_idx in range(n_cams):
        print(f"updating rot mtx for cam_idx={cam_idx}")
        
        this_rot_mtx = mat4['params'][cam_idx,0]['r'][0,0]
        this_rot_mtx = transform @ this_rot_mtx
        mat4['params'][cam_idx,0]['r'][0,0] = this_rot_mtx
    
    
    # for cam_idx in range(n_cams):
    #     print(f"updating all 3d pts in labelData[cam_idx] -> data_3d for cam_idx={cam_idx}")
        
    #     dat = mat4['labelData'][cam_idx,0][0,0]['data_3d']  # old shape: (35, 69)
    #     dat = dat.reshape(n_samples, n_joints, 3) # new shape: (35, 23, 3)
    #     for joint_idx in range(n_joints):
    #         # print(f"updating on joint_idx={joint_idx}")
    #         joint_dat = dat[:,joint_idx,:] #shape: (75,3)
    #         joint_dat = (transform @ joint_dat.T).T
    #         dat[:,joint_idx,:] = joint_dat
    #     mat4['labelData'][cam_idx,0][0,0]['data_3d'] = dat.reshape(n_samples, 3*n_joints)
    
    if saveto is not None:
        print(f"Saving update mat file as: {saveto}")
        savemat(saveto, mat4)
    else:
        print("Not saving file because saveto not specified")

def update_dannce_mat_file(filepath,saveto=None):
    print(f"update_dannce_mat_file: {filepath}")

    mat4 = loadmat(filepath)
    
    if 'lastchange' in mat4 and mat4['lastchange'] == LASTCHANGE_STRING:
        raise Exception("SKIPPING BECAUSE UPDATE WAS ALREADY APPLIED")

    save_backup_samedir(filepath)
    
    mat4['lastchange'] = LASTCHANGE_STRING
    
    n_cams = mat4['labelData'].shape[0]
    n_samples = mat4['labelData'][0,0][0,0]['data_3d'].shape[0]
    n_joints =  mat4['labelData'][0,0][0,0]['data_3d'].shape[1]//3
    
    print(f"n_cams={n_cams}, n_samples={n_samples}, n_joints={n_joints}")
    
    # update camera parameters (rotation mtx) for each camera
    # mat4['camParams'].shape = (6,1)
    for cam_idx in range(n_cams):
        print(f"updating rot mtx for cam_idx={cam_idx}")
        
        this_rot_mtx = mat4['params'][cam_idx,0]['r'][0,0]
        this_rot_mtx = transform @ this_rot_mtx
        mat4['params'][cam_idx,0]['r'][0,0] = this_rot_mtx
    
    
    for cam_idx in range(n_cams):
        print(f"updating all 3d pts in labelData[cam_idx] -> data_3d for cam_idx={cam_idx}")
        
        dat = mat4['labelData'][cam_idx,0][0,0]['data_3d']  # old shape: (35, 69)
        dat = dat.reshape(n_samples, n_joints, 3) # new shape: (35, 23, 3)
        for joint_idx in range(n_joints):
            # print(f"updating on joint_idx={joint_idx}")
            joint_dat = dat[:,joint_idx,:] #shape: (75,3)
            joint_dat = (transform @ joint_dat.T).T
            dat[:,joint_idx,:] = joint_dat
        mat4['labelData'][cam_idx,0][0,0]['data_3d'] = dat.reshape(n_samples, 3*n_joints)
    
    if saveto is not None:
        print(f"Saving update mat file as: {saveto}")
        savemat(saveto, mat4)
    else:
        print("Not saving file because saveto not specified")


def update_com3d_file(filepath, saveto=None):
    print(f"update_com3d_file: {filepath}")
    mat5 = loadmat(filepath)

    if 'lastchange' in mat5 and mat5['lastchange'] == LASTCHANGE_STRING:
        raise Exception("SKIPPING BECAUSE UPDATE WAS ALREADY APPLIED")

    save_backup_samedir(filepath)

    n_samples = mat5['com'].shape[0]
    n_joints =  1 # always 1 for COMs
    
    
    # update 3d points (labeld 3d-space)
    # format is [x y z x2 y2 z2 ... xN yN zN] for each row
    dat = mat5['com'] # old shape: (90000, 3)
    dat = dat.reshape(n_samples, n_joints, 3) # new shape: (90000, 1, 3)
    print("fixing all 3d points for #samples={n_samples}")
    for joint_idx in range(n_joints):
        print(f"updating on joint_idx={joint_idx}")
        joint_dat = dat[:,joint_idx,:] #shape: (75,3)
        joint_dat = (transform @ joint_dat.T).T
        dat[:,joint_idx,:] = joint_dat
    mat5['com'] = dat.reshape(n_samples, 3*n_joints)
    mat5['lastchange'] = LASTCHANGE_STRING

    if saveto is not None:
        print(f"Saving update mat file as: {saveto}")
        savemat(saveto, mat5)
    else:
        print("Not saving file because saveto not specified")


**Note: label3d working files are commented out**

In [4]:
# find the following:
#   1. dannce.mat
#   2. label3d working files (_Label3D.mat)
#   3. calibration (hires_camX_params.mat) files
#   4. com3d.mat files

# specify foldes to do the glob search on:

base_paths = [ 
    "/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547"
    # below 3 need to process from strach.
    # '/home/lq53/mir_data/24summ/2024_06_26/1686940_left', #validation set
    # '/home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2', 
    # '/home/lq53/mir_data/24summ/2024_07_12/240605PMC1_right_hole_11_27',
    # # labels
    # "/home/lq53/mir_data/24summ/2024_07_10/1691485_no_hole_habituation_13_59",
    # "/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFF1505",
    # "/home/lq53/mir_data/24summ/2024_07_10/1691485_left_hole_saline_10_35",
    # "/home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation",
    # "/home/lq53/mir_data/24summ/2024_07_19/240605PMC_window2_right2holes_12_14",
    # #below are my labels.
    # # "/home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation",
    # # "/home/lq53/mir_data/24summ/2024_07_12/240605PMC1_right_hole_11_27" #this one did not work
    # "/home/lq53/mir_data/24summ/2024_07_16/1691485RMHBN1405",
    # "/home/lq53/mir_data/24summ/2024_07_15/1691485RMPBS1659",
    # "/home/lq53/mir_data/24summ/2024_07_19/240605PMC_window2_right2holes_11_30",
    # "/home/lq53/mir_data/24summ/2024_07_15/1691485RMHBN1425",
    # "/home/lq53/mir_data/24summ/2024_07_15/1691485RMPBF1531"
    # 'G:\\Videos\\6cam\\lq53\\Mir_intrinsics\\pyxy3d_noT'
    # '/n/olveczky_lab_tier1/Lab/dannce_rig2/data/M1-M7_photometry/Alone', SYNTAX_ERROR_UPDATE_THIS_WITH_YOUR_OWN_FILES
    # '/n/holylabs/LABS/olveczky_lab/Lab/dannce-dev/hannah-data/COM_DANNCE_TRAINING'
]

dannce_mat_files = []
calibration_files = []
com3d_files = []

for base_path in base_paths:
    for p in Path(base_path).glob("label*/df_*_dannce.mat"): #**/ 
        if p.name.startswith("._"): 
            continue
        dannce_mat_files.append(str(p))

    for p in Path(base_path).glob("*_dannce.mat"): #**/calibration/
        if p.name.startswith("._"):
            continue
        calibration_files.append(str(p))    

    # for p in Path(base_path).glob("hires_cam*_params.mat"): #**/calibration/
    #     if p.name.startswith("._"):
    #         continue
    #     calibration_files.append(str(p))

    # for p in Path(base_path).glob("**/*com3d.mat"):
    #     if p.name.startswith("._"):
    #         continue
    #     com3d_files.append(str(p))


dannce_mat_files = sorted(dannce_mat_files)
calibration_files = sorted(calibration_files)
com3d_files = sorted(com3d_files)

print("\nDANNCE Mat files")
pp(dannce_mat_files)

print("\nCalibration Files")
pp(calibration_files)

print("\nCom3D Files")
pp(com3d_files)

allfiles = []
allfiles.extend(dannce_mat_files)
allfiles.extend(calibration_files)
allfiles.extend(com3d_files)





DANNCE Mat files
['/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/label/df_converted_mir_com_240711_1547_20240711_235440_Label3D_dannce.mat']

Calibration Files
['/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat']

Com3D Files
[]


In [5]:
# UPDATE THIS TO MAKE PATHS RELATIVE TO SOME BASE PATH
# e.g. /n/holylabs/LABS/olveckzy/Alone/M1/dannce.mat -> ./Alone/M1/dannce.mat
# allfiles = list(map(lambda x: x.replace(r"/net/holy-nfsisilon/ifs/rc_labs/olveczky_lab_tier1/Lab/dannce_rig2/data/M1-M7_photometry", "."), allfiles))
allfiles = list(map(lambda x: x.replace(r"/home/lq53/mir_data/24summ", "."), allfiles)) #2024_06_28/1686941_left_right_2

# create rsync file list
with open("file-list_3", "wt") as f:
    for p in allfiles:
        f.write(f"{p}\n")

####
# You can use RSYNC to create a clone of files listed in allfiles
### HOWEVER YOU NEED TO DO THIS 1x BASEPATH AT A TIME ###
### e.g. comment out other so that base_paths list len is 1
### and re-run above code and rsync command for each

### EXAMPLE RSYNC COMMAND
# rsync -avR --files-from=./file-list /n/olveczky_lab_tier1/Lab/dannce_rig2/data/M1-M7_photometry ./backup-files-rot-fix
# !rsync -avR --files-from=./file-list G:\\Videos\\6cam\\lq53\\Mir_intrinsics\\pyxy3d_noT ./backup-files-rot-fix
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2 /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/backup-files-rot-fix
### NOTE ON RSYNC ARGS:
### rsync -avR --files-from=FILE_LIST RSYNC_SRC RSYNC_DEST
### --avR : use archive mode, print extra output (verbose), and build relative tree (relative to dot "." in file list)
### --files-from=FILE_LIST : FILE_LIST is a text file with where each line is a file to copy, relative to RSYNC_SRC
### RSYNC_SRC : directory to look for files-from within. This is also where the dest file paths are relative to.
### RSYNC_DEST : directory where the file copies will be stored.

In [55]:
allfiles

['./2024_06_28/1686941_left_right_2/label/df_converted_168941B2_20240628rec_Arnavcom_label_20240711_164802_Label3D_dannce.mat',
 './2024_07_03/1691486_left_right_habituation/labels/df_converted_arnav_dannce_20_20240719_151827_Label3D_dannce.mat',
 './2024_07_03/1691486_left_right_habituation/labels/df_converted_mir_dannce_30_20240723_155236_Label3D_dannce.mat',
 './2024_07_03/1691486_left_right_habituation/labels/df_mir_com_26_20240807_210217_Label3D_dannce.mat',
 './2024_07_03/1691486_left_right_habituation/labels/df_mir_com_30_rand_20240815_162122_Label3D_dannce.mat',
 './2024_07_10/1691485_left_hole_saline_10_35/labels/df_converted_Arnav_dannce_20_20240718_155101_Label3D_dannce.mat',
 './2024_07_10/1691485_no_hole_habituation_13_59/label_com/df_converted_arnav_dannce_20240717_095239_Label3D_dannce.mat',
 './2024_07_10/1691485_no_hole_habituation_13_59/label_com/df_converted_mir_com_labels_20240716_183143_Label3D_dannce.mat',
 './2024_07_10/1691485_no_hole_habituation_13_59/label_com

In [6]:
!rsync -avR --files-from=./file-list_3 /home/lq53/mir_data/24summ ./backup-files-rot-fix_3


building file list ... done
created directory ./backup-files-rot-fix_3
./
2024_07_11/
2024_07_11/1691485BMCFS1547/
2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat
2024_07_11/1691485BMCFS1547/label/
2024_07_11/1691485BMCFS1547/label/df_converted_mir_com_240711_1547_20240711_235440_Label3D_dannce.mat

sent 387,189,673 bytes  received 113 bytes  774,379,572.00 bytes/sec
total size is 387,094,768  speedup is 1.00


In [28]:
# # Sync file from the first path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2 /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/backup-files-rot-fix

# # Sync file from the second path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation /home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation/backup-files-rot-fix

# # Sync file from the third path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_10/1691485_left_hole_saline_10_35 /home/lq53/mir_data/24summ/2024_07_10/1691485_left_hole_saline_10_35/backup-files-rot-fix

# # Sync file from the fourth path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_10/1691485_no_hole_habituation_13_59 /home/lq53/mir_data/24summ/2024_07_10/1691485_no_hole_habituation_13_59/backup-files-rot-fix

# # Sync file from the fifth path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFF1505 /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFF1505/backup-files-rot-fix

# # Sync file from the sixth path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_15/1691485RMHBN1425 /home/lq53/mir_data/24summ/2024_07_15/1691485RMHBN1425/backup-files-rot-fix

# # Sync file from the seventh path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_15/1691485RMPBF1531 /home/lq53/mir_data/24summ/2024_07_15/1691485RMPBF1531/backup-files-rot-fix

# # Sync file from the eighth path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_15/1691485RMPBS1659 /home/lq53/mir_data/24summ/2024_07_15/1691485RMPBS1659/backup-files-rot-fix

# # Sync file from the ninth path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_16/1691485RMHBN1405 /home/lq53/mir_data/24summ/2024_07_16/1691485RMHBN1405/backup-files-rot-fix

# # Sync file from the tenth path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_19/240605PMC_window2_right2holes_11_30 /home/lq53/mir_data/24summ/2024_07_19/240605PMC_window2_right2holes_11_30/backup-files-rot-fix

# # Sync file from the eleventh path
# !rsync -avR --files-from=./file-list /home/lq53/mir_data/24summ/2024_07_19/240605PMC_window2_right2holes_12_14 /home/lq53/mir_data/24summ/2024_07_19/240605PMC_window2_right2holes_12_14/backup-files-rot-fix

# # Continue similarly for the rest of the paths...


building file list ... 
rsync: [sender] link_stat "/home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/.2024_06_28/1686941_left_right_2/df_converted_pos_synced_1686941_left_right_2_2024_06_28_1686941_left_label3d_dannce.mat" failed: No such file or directory (2)
rsync: [sender] link_stat "/home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/.2024_07_03/1691486_left_right_habituation/df_converted_pos_synced_1691486_left_right_habituation_calib_before_label3d_dannce.mat" failed: No such file or directory (2)
rsync: [sender] link_stat "/home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/.2024_07_10/1691485_left_hole_saline_10_35/df_converted_pos_synced_1691485_left_hole_saline_10_35_calib_before_label3d_dannce.mat" failed: No such file or directory (2)
rsync: [sender] link_stat "/home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/.2024_07_10/1691485_no_hole_habituation_13_59/df_converted_pos_synced_1691485_no_hole_habituation_13_59_calib_before_label3d_dannce.ma

In [23]:
allfiles

['/home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/df_converted_pos_synced_1686941_left_right_2_2024_06_28_1686941_left_label3d_dannce.mat',
 '/home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation/df_converted_pos_synced_1691486_left_right_habituation_calib_before_label3d_dannce.mat',
 '/home/lq53/mir_data/24summ/2024_07_10/1691485_left_hole_saline_10_35/df_converted_pos_synced_1691485_left_hole_saline_10_35_calib_before_label3d_dannce.mat',
 '/home/lq53/mir_data/24summ/2024_07_10/1691485_no_hole_habituation_13_59/df_converted_pos_synced_1691485_no_hole_habituation_13_59_calib_before_label3d_dannce.mat',
 '/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFF1505/df_converted_pos_synced_1691485BMCFF1505_calib_before_label3d_dannce.mat',
 '/home/lq53/mir_data/24summ/2024_07_15/1691485RMHBN1425/df_converted_pos_synced_1691485RMHBN1425_calib_before_label3d_dannce.mat',
 '/home/lq53/mir_data/24summ/2024_07_15/1691485RMPBF1531/df_converted_pos_synced_1691485RMPBF1531_c

In [50]:
import shutil
from pathlib import Path

def revert_backup_samedir(src_filepath):
    """Revert to the latest .old backup and remove all .old backups in the same directory."""
    original_file = Path(src_filepath)
    assert original_file.exists(), f"{src_filepath} does not exist."

    # Find all backup files for the original file
    backup_files = sorted(Path(f"{src_filepath}.old").parent.glob(f"{original_file.name}.old"), #old*, but i want only .old just in case
                          key=lambda x: x.stat().st_mtime, reverse=True)
    
    if not backup_files:
        raise FileNotFoundError("No backup files found.")
    
    # Use the most recent backup
    latest_backup = backup_files[0]
    print(f"Reverting {src_filepath} to {latest_backup}")

    # Copy the latest backup to the original file
    shutil.copy2(latest_backup, src_filepath)
    print(f"Reverted {src_filepath} to {latest_backup}")

    # Remove all .old backups
    for backup in backup_files:
        try:
            backup.unlink()
            print(f"Removed backup: {backup}")
        except Exception as e:
            print(f"Failed to remove backup: {backup}. Reason: {e}")

# Example usage
# revert_backup_samedir('/home/lq53/mir_data/text_orig.txt')

FileNotFoundError: No backup files found.

In [7]:
### RUN FIX ON EACH LIST OF FILES ###
######################################################################
### I WOULD UN-COMMENT AND RUN ONE GROUP AT A TIME 
### IF THERE IS AN ERROR WITH ONE OF THE FILES, YOU CAN CONTINUE AFTER
###    A CERTAIN INDEX. E.G. 5
###
### for i,f in enumerate(calibration_files[5:]): ...
#######################################################################

for i,f in enumerate(calibration_files):
    print (f"calib_file: [{i}]:" ,f)
    # update_hires_file(f, f)
    try:
        # revert_backup_samedir(f)
        update_calib_for_dannce(f, f)  # for calibs
    except Exception as e:
        print(f"Skipping file {f} due to: {e}")
        continue    
    # update_calib_for_dannce(f,f) #for calibs
        
for i,f in enumerate(dannce_mat_files):
    print (f"dannce.mat: [{i}]:" ,f)
    #in case still some calib files are taken in as label files...
    # if "pos" in f:
    #     try:
    #         update_calib_for_dannce(f, f)  # for calibs
    #     except Exception as e:
    #         print(f"Skipping file {f} due to: {e}")
    #         continue
        # update_calib_for_dannce(f,f) #for calibs
    # else:
    try:
        # revert_backup_samedir(f)
        update_dannce_mat_file(f, f)  # for calibs
    except Exception as e:
        print(f"Skipping file {f} due to: {e}")
        continue
        
        # update_dannce_mat_file(f,f) # for labels

# for i,f in enumerate(com3d_files):
#     print (f"c3d: [{i}]:" ,f)
#     try:
#         update_com3d_file(f, f)  # for calibs
#     except Exception as e:
#         print(f"Skipping file {f} due to: {e}")
#         continue
    # update_com3d_file(f,f)

calib_file: [0]: /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat
update_dannce_mat_file: /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat
BACKUP src=/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat dest=/home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat.old
updating rot mtx for cam_idx=0
Saving update mat file as: /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/df_converted_pos_synced_1691485BMCFS1547_calib_before_label3d_dannce.mat
dannce.mat: [0]: /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/label/df_converted_mir_com_240711_1547_20240711_235440_Label3D_dannce.mat
update_dannce_mat_file: /home/lq53/mir_data/24summ/2024_07_11/1691485BMCFS1547/label/df_convert

In [42]:
# example code to verify all z-values are positive in COM files

for fname in com3d_files:
    mtmp = loadmat(fname)
    # print("KEYS", mtmp.keys())
    n_samples = mtmp['com'].shape[0]
    print("FNAME: ", fname)
    sum_gt_z = sum(mtmp['com'][:,2] > 0)
    print(f"COM's with z-coord > zero {sum_gt_z}/{n_samples}\n")



FNAME:  /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/COM/predict_results/com3d.mat
COM's with z-coord > zero 53888/53999

FNAME:  /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/COM_240716weights/predict_results/com3d.mat
COM's with z-coord > zero 53803/53999

FNAME:  /home/lq53/mir_data/24summ/2024_06_28/1686941_left_right_2/COM_240806weights/predict_results/com3d.mat
COM's with z-coord > zero 53510/53997

FNAME:  /home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation/COM/predict_results/com3d.mat
COM's with z-coord > zero 35994/35994

FNAME:  /home/lq53/mir_data/24summ/2024_07_03/1691486_left_right_habituation/COM_240716weights/predict_results/com3d.mat
COM's with z-coord > zero 35993/35993

FNAME:  /home/lq53/mir_data/24summ/2024_07_10/1691485_left_hole_saline_10_35/COM/0716new/com3d.mat
COM's with z-coord > zero 35981/35992

FNAME:  /home/lq53/mir_data/24summ/2024_07_10/1691485_left_hole_saline_10_35/COM/predict_results/com3d.mat
COM's with z-